In [4]:
import cv2

# Cargar el clasificador Haarcascade para automóviles
car_cascade = cv2.CascadeClassifier('cars.xml')

# Iniciar la captura de video desde la cámara (puedes ajustar el número de la cámara según tus necesidades)
cap = cv2.VideoCapture(0)

while True:
    # Leer un fotograma de la cámara
    ret, frame = cap.read()

    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar automóviles en la imagen
    cars = car_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3)


    # Dibujar rectángulos alrededor de los automóviles detectados
    for (x, y, w, h) in cars:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

    # Mostrar la imagen resultante
    cv2.imshow('Car Detection', frame)

    # Romper el bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la captura de video y cerrar la ventana
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.8.1) :-1: error: (-5:Bad argument) in function 'CascadeClassifier'
> Overload resolution failed:
>  - CascadeClassifier() takes at most 1 argument (2 given)


In [1]:
#con machine learningimport cv2
import cv2
import numpy as np

# Cargar la red YOLO preentrenada
net = cv2.dnn.readNet('./../../../venv/yolov3.weights','./../../../venv/yolov3.cfg')
layer_names = net.getUnconnectedOutLayersNames()

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    # Obtener las dimensiones de la imagen
    height, width, _ = frame.shape

    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Obtener las detecciones
    detections = net.forward(layer_names)

    # Procesar las detecciones
    
    for detection in detections[0]:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > 0.5 and class_id == 2:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Coordenadas de la esquina superior izquierda
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            # Dibujar rectángulo alrededor del carro detectado
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

    # Mostrar la imagen resultante
    cv2.imshow('Car Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
